# Dados dos professores da UDESC

Descobrindo quais serão os possíveis professores que serão os meus professores

In [1]:
# Biblioteca utilizadas
from bs4 import BeautifulSoup
import requests
from time import sleep
from tqdm import tqdm

In [2]:
# Obtendo dados do site da UDESC na página sobre os professores
url = requests.get("https://www.udesc.br/professores").text

soup = BeautifulSoup(url, 'html5lib')

In [3]:
def num_maximo_professores():
    """Descobre o número máximo de professores"""
    
    return len([span for span in soup('span', {'id': 'nome'})])

In [4]:
def unidades_de_ensino():
    """Descobre quais são as unidade de ensino existentes"""
    
    unidades_de_ensino = set(div.text 
                             for div in soup('div', {'id': 'txtUnidadeDeEnsino'}))
    
    return [unidade.replace('Unidade de ensino:\xa0\xa0', '').strip() for unidade in unidades_de_ensino]

In [5]:
def escrevendo_unidades_ensino():
    with open('unidades_de_ensino.csv', 'wt+') as f:
        for n, unidade in enumerate(unidades_de_ensino()):
            if n != len(unidades_de_ensino()) -1:
                f.write('{}\n'.format(unidade))
            else:
                f.write('{}'.format(unidade))
            

In [6]:
def filtro_por_unidade_de_ensino(nome, url_lista_professores, unidade_de_ensino, filtro):
    """Filtra os professores por unidade de ensino"""
    
    filtro_nome = []
    filtro_url = []
    filtro_unidade = []
    
    for n, unidade in enumerate(unidade_de_ensino):
        if unidade == filtro:
            filtro_nome.append(nome[n])
            filtro_url.append(url_lista_professores[n])
            filtro_unidade.append(unidade_de_ensino[n])
    return filtro_nome, filtro_url, filtro_unidade    

In [7]:
def get_dados_professores_lista(num_dados=5, filtro=None):
    """
    Coleta os dados principais encontrados na página da lista
    de professores
    """
    
    # Caso queira pesquisar todos os professores
    # Cololoque no campo 'num_dados' recebendo 'ALL'
    if num_dados == "ALL":
        num_dados = num_maximo_professores()
        
    # Recomendasse fortemente filtrar os dados por unidade de ensino
    # caso queira pesquisar todos os professores 
    
    udesc_url = "https://www.udesc.br"
    
    span_nome_lista_professores = [span.a.text 
                                   for span in soup('span', {'id': 'nome'})]

    span_url_lista_professores = [udesc_url + span.a.get('href') 
                                  for span in soup('span', {'id': 'nome'})]

    div_unidade_de_ensino_lista_professor = [div.text 
                                             for div in soup('div', {'id': 'txtUnidadeDeEnsino'})]
    
    div_unidade_de_ensino_lista_professor = [unidade.replace('Unidade de ensino:\xa0\xa0', '').strip() 
                                             for unidade in div_unidade_de_ensino_lista_professor]
    
    # Filtro
    if filtro != None:
        nome, url_site, unidade_ensino = filtro_por_unidade_de_ensino(span_nome_lista_professores,
                                                                     span_url_lista_professores,
                                                                     div_unidade_de_ensino_lista_professor,
                                                                     filtro=filtro)
       
        print('Com filtro')
        
        # Caso queira pesquisar todos os professores de uma unidade
       
        
        dados_professores = {'NOME' : nome[:num_dados],
                  'URL SITE' : url_site[:num_dados],
                  'UNIDADE DE ENSINO' : unidade_ensino[:num_dados],
                    'DEPARTAMENTO': None,
                    'TITULAÇÃO': None}
        
    else:
        print('Sem filtro')
        dados_professores = {'NOME' : span_nome_lista_professores[:num_dados],
                      'URL SITE' : span_url_lista_professores[:num_dados],
                      'UNIDADE DE ENSINO' : div_unidade_de_ensino_lista_professor[:num_dados],
                        'DEPARTAMENTO': None,
                        'TITULAÇÃO': None}
    
    return dados_professores

In [8]:
def pesquisando_professores(url_professores, time_spleep=5):
    
    lista_departamento = []
    lista_titulacao = []
     
    for url in tqdm(url_professores):
        
        url_professor = requests.get(url).text
        soup_professor = BeautifulSoup(url_professor, 'html5lib')
        
        div_professor = [div.text 
                         for div in soup_professor('div', {'class': 'campo_dado'})]
        
        departamento = str(div_professor[0]).split()[3:]
        departamento = " ".join(departamento)
        
        titulacao = str(div_professor[1]).split()[1:]
        titulacao = " ".join(titulacao)
        
        lista_departamento.append(departamento)
        lista_titulacao.append(titulacao)
        
        sleep(time_spleep) 

        
    return lista_departamento, lista_titulacao      

In [9]:
def criando_arquivo(dados_professores, nome_arquivo='dados_professores.csv'):
    with open("dados_brutos/" + nome_arquivo, 'wt+') as f:
        # Chave 
        for key in dados_professores.keys():
            if key != "TITULAÇÃO":
                f.write("{};".format(key))
            else:
                f.write("{}\n".format(key))
        
        # Valor
        for n, _ in enumerate(dados_professores['NOME']):
            f.write("{};{};{};{};{}\n".format(
            dados_professores['NOME'][n],
            dados_professores['URL SITE'][n],
            dados_professores['UNIDADE DE ENSINO'][n],
            dados_professores['DEPARTAMENTO'][n],
            dados_professores['TITULAÇÃO'][n])) 

In [10]:
def get_professores(num_dados=5, filtro=None, time_sleep=5, nome_arquivo='dados_professores.csv'):
    """
    Coleta os dados principais encontrados na página pessoal de cada
    professor
    """
    
    professores = get_dados_professores_lista(num_dados, filtro)
    
    # Se quiser pegar todos os professores de um determinado filtro
    if num_dados == "ALL":
        num_dados = len(professores['NOME'])
        print(num_dados)
    
    departamento, titulacao = pesquisando_professores(professores['URL SITE'], time_sleep)
    
    professores['DEPARTAMENTO'], professores['TITULAÇÃO'] = departamento[:num_dados], titulacao[:num_dados]
    
    # Escrevendo os dados 
    criando_arquivo(professores, nome_arquivo)